model to predict the review rating from its text and make a sort of sentiment analysis using nltk.

First we import the basic modules required for working with our data like paqdas for reading and numpy for numerical operations on Matrices.

we also select and import the reuired classifier, here being a MultilayerPerceptron

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)



We then move to read the data from a csv file containing a list of over 34000 consumer reviews for Amazon products like the Kindle, Fire TV Stick, and more provided by Datafiniti Product Database. The dataset includes basic product information, rating, review text, and more for each product.

We also print two rows to see what data is present along with to help us recognise the usable parameters.

In [2]:
df = pd.read_csv('1429_1.csv')
df.head(2)

C:\Users\Admin\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3057: DtypeWarning: Columns (1,10) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,id,name,asins,brand,categories,keys,manufacturer,reviews.date,reviews.dateAdded,reviews.dateSeen,...,reviews.doRecommend,reviews.id,reviews.numHelpful,reviews.rating,reviews.sourceURLs,reviews.text,reviews.title,reviews.userCity,reviews.userProvince,reviews.username
0,AVqkIhwDv8e3D1O-lebb,"All-New Fire HD 8 Tablet, 8 HD Display, Wi-Fi,...",B01AHB9CN2,Amazon,"Electronics,iPad & Tablets,All Tablets,Fire Ta...","841667104676,amazon/53004484,amazon/b01ahb9cn2...",Amazon,2017-01-13T00:00:00.000Z,2017-07-03T23:33:15Z,"2017-06-07T09:04:00.000Z,2017-04-30T00:45:00.000Z",...,True,NaN,0.0,5.0,http://reviews.bestbuy.com/3545/5620406/review...,This product so far has not disappointed. My c...,Kindle,NaN,NaN,Adapter
1,AVqkIhwDv8e3D1O-lebb,"All-New Fire HD 8 Tablet, 8 HD Display, Wi-Fi,...",B01AHB9CN2,Amazon,"Electronics,iPad & Tablets,All Tablets,Fire Ta...","841667104676,amazon/53004484,amazon/b01ahb9cn2...",Amazon,2017-01-13T00:00:00.000Z,2017-07-03T23:33:15Z,"2017-06-07T09:04:00.000Z,2017-04-30T00:45:00.000Z",...,True,NaN,0.0,5.0,http://reviews.bestbuy.com/3545/5620406/review...,great for beginner or experienced person. Boug...,very fast,NaN,NaN,truman


Now we import the NLTK library which will help us perform the required sentiment analysis and natural language text processing on the coloumn 'reviews.text'

We also import the required libraries from nltk like word_tokenise to tokenise the text and stopwords corpus to be filtered out from it.

Post that we import featureextraction and pre/post processing libraies like CountVecorizer which will help us extract our n-grams later on, train_test_split to divide the training and test data, and also finally various matrics to evaluate the performance of the model.

In [ ]:
import nltk 
from nltk import word_tokenize
from nltk.corpus import stopwords
import re
import string
from collections import Counter
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import normalize
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from sklearn.naive_bayes import GaussianNB
nltk.download('stopwords')


Our text contains words that will not contribute to our cause and hence we perform the nessecary data cleaning by defining the functions for the below filtering actions,

- Punctuation removal
- Stop Words removal - e.g. ‘the’, ‘is’, ‘are’. There is no universal list of stop words in nlp research, however the nltk module contains a list of stop words.
- remove the standard Amazon string if present which says if teh review was helpful or not.


In [ ]:
def removePunctuation(x):
    x = x.lower()
    x = re.sub(r'[^\x00-\x7f]',r' ',x)
    return re.sub("["+string.punctuation+"]", " ", x)

stops = set(stopwords.words("english"))
def removeStopwords(x):
    final_words = [word for word in x.split() if word not in stops]
    return " ".join(final_words)
def removeAmazonQoute(x):
    return re.sub(r'[0-9]+ people found this helpful\. Was this review helpful to you Yes No', "", x)

we now define our target vector which is the 'reviews.ratings' coloumn

In [ ]:
df['reviews.rating']=df['reviews.rating'].fillna(0)
target = df['reviews.rating']

the input matrice for review text will be now created and cleaned up to contain only essential usable text, using the functions we defined earlier.

In [ ]:
reviews = [sent if type(sent)==str else "" for sent in df['reviews.title'].values]
reviews = [removeAmazonQoute(sent) for sent in reviews]
reviews = [removeStopwords(sent) for sent in reviews]
reviews = [removePunctuation(sent) for sent in reviews]

Our Input data is now ready hence we now could use a CountVectorizer to turn each word of our reviews dataset into a number and count its occurrences in the sentence.

This technique is called Bag of Words https://en.wikipedia.org/wiki/Bag-of-words_model. However in this case we are going to use an alternative model called N-Gram Model https://en.wikipedia.org/wiki/N-gram in which each sequence of three words is turned into a number and its occurrencies counted. This model does not only consider the word itself but the relationship with its neighbour words.

In [ ]:
cv = CountVectorizer(stop_words=None,token_pattern=r'\b\w\w+-?\w+\b',ngram_range=(1, 3))
fcv = cv.fit_transform(reviews)

We now can finally convert the n-gram vectorised data 'fcv' into an array to get the final Input.

We then use the final input 'rev_array' and the target array defined earlier to create the training and test data, we use the train_test_split library from scikit-learn for this purpose.

In [ ]:
rev_array = fcv.toarray()
X_train, X_test, y_train, y_test= train_test_split(rev_array, target, test_size=0.33, random_state=2)

We are now ready to train our model and for that we use a MultiLayer Perceptron with relu activation and adaptive learning rate.

In [ ]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(n_estimators=100,criterion='entropy')
y_pred = clf.fit(X_train, y_train).predict(X_test)

we now evaluate our model performace for its predicted ratings based on review text against the actual ratings

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, y_pred)

We have recieved a '0.67871212121212122' accuracy score for our model. 